# Hooks: Customizing Agent Behavior

In this notebook, we'll explore how to use hooks to intercept and customize agent behavior at key lifecycle points.

## Setup

Configure the environment:

In [ ]:
# Setup for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✓ Notebook environment configured")

In [ ]:
import os

# Verify API key
api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key found (length: {len(api_key)} characters)")
else:
    print("✗ API key not found. Please set ANTHROPIC_API_KEY environment variable.")

## Why Hooks?

Hooks allow you to:
- **Monitor** agent behavior and tool usage
- **Log** queries, responses, and tool calls
- **Control** tool execution with approval workflows
- **Transform** messages before/after processing
- **Integrate** with external systems (databases, analytics, alerts)

### Available Hooks

| Hook | When It Fires | Use Case |
|------|---------------|----------|
| `on_query_start` | Before query processing | Log start, set up context |
| `on_query_end` | After query completes | Log results, cleanup |
| `on_tool_use` | Before tool execution | Approval workflows, validation |
| `on_tool_result` | After tool execution | Log outcomes, error handling |
| `on_message` | For each message | Real-time streaming, UI updates |

## Example 1: Basic Logging Hook

Let's start with a simple logging hook:

In [ ]:
from claude_agent_sdk import query, ClaudeAgentOptions
from datetime import datetime

# Create a simple logger
class QueryLogger:
    def __init__(self):
        self.logs = []
    
    def on_query_start(self, prompt):
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "event": "query_start",
            "prompt": prompt
        }
        self.logs.append(log_entry)
        print(f"📝 [LOG] Query started: {prompt[:50]}...")
    
    def on_query_end(self, result):
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "event": "query_end",
            "result": str(result)[:100]
        }
        self.logs.append(log_entry)
        print(f"📝 [LOG] Query completed")
    
    def print_logs(self):
        print("\n" + "=" * 60)
        print("QUERY LOGS")
        print("=" * 60)
        for log in self.logs:
            print(f"{log['event']}: {log['timestamp']}")
            if 'prompt' in log:
                print(f"  Prompt: {log['prompt'][:80]}...")
            if 'result' in log:
                print(f"  Result: {log['result'][:80]}...")

# Use the logger
logger = QueryLogger()

async def with_logging():
    async for msg in query(
        prompt="What files are in the current directory?",
        options=ClaudeAgentOptions(
            hooks={
                "on_query_start": logger.on_query_start,
                "on_query_end": logger.on_query_end
            }
        )
    ):
        pass  # Process messages silently
    
    # Print the logs
    logger.print_logs()

await with_logging()

### What Happened?

The hooks logged:
1. When the query started (with the prompt)
2. When the query ended (with the result)

This is useful for:
- Debugging agent behavior
- Tracking usage patterns
- Building audit trails

## Example 2: Tool Usage Monitoring

Let's track which tools the agent uses:

In [ ]:
from collections import Counter

class ToolMonitor:
    def __init__(self):
        self.tool_usage = Counter()
        self.tool_calls = []
    
    def on_tool_use(self, tool_name, tool_input):
        self.tool_usage[tool_name] += 1
        self.tool_calls.append({
            "tool": tool_name,
            "input": tool_input,
            "timestamp": datetime.now()
        })
        print(f"🔧 [MONITOR] Tool used: {tool_name}")
    
    def on_tool_result(self, tool_name, result, is_error):
        status = "ERROR" if is_error else "SUCCESS"
        print(f"📤 [MONITOR] Tool {tool_name}: {status}")
    
    def print_summary(self):
        print("\n" + "=" * 60)
        print("TOOL USAGE SUMMARY")
        print("=" * 60)
        print(f"Total tool calls: {sum(self.tool_usage.values())}")
        print("\nTool breakdown:")
        for tool, count in self.tool_usage.most_common():
            print(f"  {tool}: {count} calls")

# Use the monitor
monitor = ToolMonitor()

async def monitor_tools():
    async for msg in query(
        prompt="Find all .py files and count the lines in the first one",
        options=ClaudeAgentOptions(
            hooks={
                "on_tool_use": monitor.on_tool_use,
                "on_tool_result": monitor.on_tool_result
            },
            permission_mode="bypassPermissions"
        )
    ):
        pass  # Process silently
    
    monitor.print_summary()

await monitor_tools()

### Why Monitor Tools?

Tool monitoring helps you:
- Understand which tools agents use most
- Identify bottlenecks or inefficiencies
- Track error rates per tool
- Optimize tool availability
- Debug agent decision-making

## Example 3: Approval Workflow

Implement a custom approval system for sensitive tools:

In [ ]:
class ApprovalWorkflow:
    def __init__(self, require_approval_for):
        self.require_approval_for = require_approval_for
        self.approved_tools = []
        self.denied_tools = []
    
    def on_tool_use(self, tool_name, tool_input):
        if tool_name in self.require_approval_for:
            print(f"\n⚠️  Tool '{tool_name}' requires approval")
            print(f"Input: {tool_input}")
            
            # In a real app, this would be an async prompt to the user
            # For demo, we'll auto-approve
            approved = True  # In real code: get user input
            
            if approved:
                print("✅ Approved")
                self.approved_tools.append(tool_name)
                return True  # Allow execution
            else:
                print("❌ Denied")
                self.denied_tools.append(tool_name)
                return False  # Block execution
        
        # Auto-approve other tools
        return True
    
    def print_summary(self):
        print("\n" + "=" * 60)
        print("APPROVAL SUMMARY")
        print("=" * 60)
        print(f"Approved: {len(self.approved_tools)}")
        print(f"Denied: {len(self.denied_tools)}")

# Create approval workflow for write operations
approval = ApprovalWorkflow(require_approval_for=["Write", "Edit", "Bash"])

async def with_approval():
    async for msg in query(
        prompt="List the Python files in this directory",
        options=ClaudeAgentOptions(
            hooks={
                "on_tool_use": approval.on_tool_use
            }
        )
    ):
        pass  # Process silently
    
    approval.print_summary()

await with_approval()

### Approval Use Cases

- **Write operations**: Prevent unwanted file modifications
- **External API calls**: Control costs and rate limits
- **Sensitive data access**: Audit who accesses what
- **Production systems**: Require human approval for risky actions

## Example 4: Cost Tracking

Track API costs across queries:

In [ ]:
class CostTracker:
    def __init__(self, budget_usd):
        self.budget_usd = budget_usd
        self.total_cost = 0.0
        self.query_costs = []
    
    def on_query_end(self, result):
        # Extract cost from result message
        if hasattr(result, 'total_cost_usd'):
            cost = result.total_cost_usd
            self.total_cost += cost
            self.query_costs.append(cost)
            
            remaining = self.budget_usd - self.total_cost
            
            print(f"\n💰 Query cost: ${cost:.4f}")
            print(f"💰 Total cost: ${self.total_cost:.4f}")
            print(f"💰 Remaining budget: ${remaining:.4f}")
            
            if remaining < 0:
                print("⚠️  WARNING: Budget exceeded!")
            elif remaining < self.budget_usd * 0.1:
                print("⚠️  WARNING: Less than 10% of budget remaining")
    
    def print_summary(self):
        print("\n" + "=" * 60)
        print("COST TRACKING SUMMARY")
        print("=" * 60)
        print(f"Budget: ${self.budget_usd:.4f}")
        print(f"Total spent: ${self.total_cost:.4f}")
        print(f"Queries: {len(self.query_costs)}")
        if self.query_costs:
            print(f"Average per query: ${sum(self.query_costs)/len(self.query_costs):.4f}")
            print(f"Most expensive query: ${max(self.query_costs):.4f}")
            print(f"Cheapest query: ${min(self.query_costs):.4f}")

# Track costs with $0.50 budget
cost_tracker = CostTracker(budget_usd=0.50)

async def track_costs():
    # Make several queries
    queries = [
        "List Python files",
        "Count lines in the first file",
        "Search for TODO comments"
    ]
    
    for i, prompt in enumerate(queries, 1):
        print(f"\n{'='*60}")
        print(f"Query {i}: {prompt}")
        print("="*60)
        
        async for msg in query(
            prompt=prompt,
            options=ClaudeAgentOptions(
                hooks={"on_query_end": cost_tracker.on_query_end},
                permission_mode="bypassPermissions"
            )
        ):
            pass  # Process silently
    
    cost_tracker.print_summary()

await track_costs()

### Cost Management Benefits

- **Budget enforcement**: Stop queries when budget is exhausted
- **Cost visibility**: Understand spending per query
- **Optimization**: Identify expensive queries to optimize
- **Forecasting**: Predict costs for production workloads

## Example 5: Message Streaming Hook

Process messages in real-time as they stream:

In [ ]:
class MessageStreamer:
    def __init__(self):
        self.message_count = 0
    
    def on_message(self, message):
        self.message_count += 1
        msg_type = type(message).__name__
        
        if msg_type == "AssistantMessage":
            # Extract text content
            if hasattr(message, 'content'):
                for block in message.content:
                    if type(block).__name__ == "TextBlock":
                        print(f"\n💬 Assistant: {block.text[:100]}...")
        
        elif msg_type == "ResultMessage":
            print(f"\n✅ Query completed after {self.message_count} messages")

streamer = MessageStreamer()

async def stream_messages():
    async for msg in query(
        prompt="Tell me about the Python files in this directory",
        options=ClaudeAgentOptions(
            hooks={"on_message": streamer.on_message},
            permission_mode="bypassPermissions"
        )
    ):
        pass  # Hook handles printing

await stream_messages()

### Streaming Use Cases

- **UI updates**: Show progress in real-time
- **Live logs**: Stream to console or log files
- **WebSocket integration**: Push updates to connected clients
- **Progress indicators**: Show spinners or progress bars

## Example 6: Combining Multiple Hooks

Use multiple hooks together for comprehensive monitoring:

In [ ]:
class ComprehensiveMonitor:
    def __init__(self):
        self.start_time = None
        self.tool_count = 0
        self.message_count = 0
        self.cost = 0.0
    
    def on_query_start(self, prompt):
        self.start_time = datetime.now()
        print(f"\n🚀 Query started at {self.start_time.strftime('%H:%M:%S')}")
        print(f"📝 Prompt: {prompt[:80]}...")
    
    def on_tool_use(self, tool_name, tool_input):
        self.tool_count += 1
        print(f"🔧 Tool #{self.tool_count}: {tool_name}")
    
    def on_message(self, message):
        self.message_count += 1
    
    def on_query_end(self, result):
        end_time = datetime.now()
        duration = (end_time - self.start_time).total_seconds()
        
        if hasattr(result, 'total_cost_usd'):
            self.cost = result.total_cost_usd
        
        print("\n" + "="*60)
        print("QUERY METRICS")
        print("="*60)
        print(f"⏱️  Duration: {duration:.2f}s")
        print(f"💬 Messages: {self.message_count}")
        print(f"🔧 Tool calls: {self.tool_count}")
        print(f"💰 Cost: ${self.cost:.4f}")
        print(f"📊 Efficiency: {self.tool_count/duration:.2f} tools/sec")

monitor = ComprehensiveMonitor()

async def comprehensive_monitoring():
    async for msg in query(
        prompt="Find all .ipynb files, count them, and tell me about the first one",
        options=ClaudeAgentOptions(
            hooks={
                "on_query_start": monitor.on_query_start,
                "on_tool_use": monitor.on_tool_use,
                "on_message": monitor.on_message,
                "on_query_end": monitor.on_query_end
            },
            permission_mode="bypassPermissions"
        )
    ):
        pass

await comprehensive_monitoring()

## Hook Best Practices

### Design
- Keep hooks **fast** - don't block query processing
- Make hooks **idempotent** - they may fire multiple times
- Use **async hooks** for I/O operations
- **Separate concerns** - one hook per responsibility

### Error Handling
- Always use try/except in hooks
- Don't let hook errors crash queries
- Log hook errors separately

### Performance
- Avoid heavy computation in hooks
- Use background tasks for slow operations
- Consider buffering for high-frequency hooks

### Security
- Validate approval decisions carefully
- Don't log sensitive data (API keys, passwords)
- Sanitize tool inputs before logging

## Exercises

Practice building hooks:

### Exercise 1: Audit Trail
Create a hook that logs:
- All queries with timestamps
- All tool uses with inputs
- All results
- Write logs to a JSON file

### Exercise 2: Rate Limiter
Build a hook that:
- Limits tool calls to 10 per minute
- Rejects excess calls with an error message
- Resets the counter every minute

### Exercise 3: Smart Approval
Create an approval hook that:
- Auto-approves Read/Glob/Grep
- Auto-denies Bash commands with `rm` or `sudo`
- Prompts user for other tools
- Logs all decisions

In [ ]:
# Exercise 1: Your solution here


In [ ]:
# Exercise 2: Your solution here


In [ ]:
# Exercise 3: Your solution here


## Key Takeaways

- **Hooks intercept agent behavior** at key lifecycle points
- **Five hook types**: query start/end, tool use/result, messages
- **Use cases**: logging, monitoring, approval, cost tracking, streaming
- **Multiple hooks** can work together for comprehensive control
- **Keep hooks fast** - don't block query processing
- **Error handling** is critical - don't let hooks crash queries

Next up: **Skills** - learn how to create reusable agent behaviors and workflows!